In [1]:
import geopandas as gpd
import pandas as pd
import folium
from branca.colormap import linear
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import openmatrix as omx

In [2]:
# List of TAZ rows to extract
taz_rows_to_extract_df = pd.DataFrame([
    [ 832, 'Bountiful'    ],
    [1629, 'Murray'       ],
    [1984, 'South Jordan' ],
    [3167, 'Spanish Fork' ],
    [2845, 'MAG Office'   ],
    [2513, 'Saratoga'      ]
], columns=['taz_id','map_label'])

taz_rows_to_extract = taz_rows_to_extract_df['taz_id'].drop_duplicates().to_list()

output_csv = "intermediate/TAZ-PA-Time.csv"

# Define the input OMX file and output CSV file
omx_files_df = pd.DataFrame([
    ['auto', 'data/input_matrix_auto.omx'],
    ['transit', 'data/input_matrix_transit.omx']
], columns=['AutoTransit','omx_filename'])
display(omx_files_df)

matrices_for_extraction_df = pd.DataFrame([
    ['auto','none',['GP_TotTime']],#,'MG_TotTime']],
    ['transit','walk',['w4time','w5time','w6time','w7time','w8time','w9time']],
    ['transit','drive',['d4time','d5time','d6time','d7time','d8time','d9time']],
], columns=['AutoTransit','AccessMode','matrices_to_get_minimum'])
display(matrices_for_extraction_df)

,AutoTransit,omx_filename
0,auto,data/input_matrix_auto.omx
1,transit,data/input_matrix_transit.omx


,AutoTransit,AccessMode,matrices_to_get_minimum
0,auto,none,[GP_TotTime]
1,transit,walk,"[w4time, w5time, w6time, w7time, w8time, w9time]"
2,transit,drive,"[d4time, d5time, d6time, d7time, d8time, d9time]"


In [3]:
# Convert TAZ rows to zero-based index
matrix_rows_to_extract = [i - 1 for i in taz_rows_to_extract]

# Initialize list to store extracted data
extracted_data = []

# Loop through OMX files
for _, row_files in omx_files_df.iterrows():
    auto_transit = row_files['AutoTransit']

    # Open OMX file
    with omx.open_file(row_files['omx_filename'], 'r') as omx_data:
        
        # Filter matrices for the current AutoTransit mode
        matrices_for_extraction_filtered_df = matrices_for_extraction_df[
            matrices_for_extraction_df['AutoTransit'] == auto_transit
        ]

        # Loop over each AutoTransit & AccessMode combination
        for _, row in matrices_for_extraction_filtered_df.iterrows():
            access_mode = row["AccessMode"]
            matrices_to_extract = row["matrices_to_get_minimum"]

            # Extract and compute the minimum values across matrices for each I-J pair
            for i in matrix_rows_to_extract:
                for j in range(omx_data[matrices_to_extract[0]].shape[1]):  # Iterate over all columns (J values)
                    
                    # Extract values across all matrices at (i, j), filtering out 0 values
                    matrix_values = [omx_data[matrix][i, j] for matrix in matrices_to_extract]
                    non_zero_values = [val for val in matrix_values if val > 0]  # Filter out 0s
                    
                    # Get the minimum value across non-zero matrices (if any remain)
                    min_value = min(non_zero_values) if non_zero_values else None
                    
                    # Append extracted data only if there's a valid minimum
                    if min_value is not None:
                        extracted_data.append([i + 1, j + 1, min_value, auto_transit, access_mode])

# Convert to DataFrame
extracted_data_df = pd.DataFrame(extracted_data, columns=['TAZ_P', 'TAZ_A', 'Value', 'AutoTransit', 'AccessMode'])
display(extracted_data_df)

,TAZ_P,TAZ_A,Value,AutoTransit,AccessMode
0,832,1,77.70,auto,none
1,832,2,77.14,auto,none
2,832,3,72.45,auto,none
3,832,4,73.51,auto,none
4,832,5,76.02,auto,none
...,...,...,...,...,...
36640,2513,3449,93.06,transit,drive
36641,2513,3453,92.17,transit,drive
36642,2513,3454,94.62,transit,drive
36643,2513,3455,92.05,transit,drive


In [4]:
# Save to CSV
extracted_data_df.to_csv(output_csv, index=False)

print(f"Extracted data saved to {output_csv}")

Extracted data saved to intermediate/TAZ-PA-Time.csv


In [5]:
travel_shed_ato_classes_df = pd.read_csv("params/NEW And OLD ATO Method (for travelsheds).csv")
travel_shed_ato_classes_df

,Method,AutoTransit,AccessMode,TTStart_Minutes,TTEnd_Minutes_Inclusive,MapClass,JobFract
0,New,auto,none,0.00,10.00,11,1
1,New,auto,none,10.00,13.50,10,0.90 - 0.99
2,New,auto,none,13.50,17.00,9,0.80 - 0.89
3,New,auto,none,17.00,20.50,8,0.70 - 0.79
4,New,auto,none,20.50,24.00,7,0.60 - 0.69
...,...,...,...,...,...,...,...
58,Old,transit,drive,17.25,20.25,5,0.40 - 0.49
59,Old,transit,drive,20.25,23.75,4,0.30 - 0.39
60,Old,transit,drive,23.75,29.25,3,0.20 - 0.29
61,Old,transit,drive,29.25,39.25,2,0.10 - 0.19


In [6]:
# Performing the join based on the Value being between TTStart_Minutes and TTEnd_Minutes
merged_df = extracted_data_df.merge(
    travel_shed_ato_classes_df,
    on=['AutoTransit','AccessMode']
).query("TTStart_Minutes < Value <= TTEnd_Minutes_Inclusive")

# Display the merged DataFrame
display(merged_df)


,TAZ_P,TAZ_A,Value,AutoTransit,AccessMode,Method,TTStart_Minutes,TTEnd_Minutes_Inclusive,MapClass,JobFract
3035,832,138,58.95,auto,none,Old,39.25,60.0,1,0.00 - 0.09
3057,832,139,58.12,auto,none,Old,39.25,60.0,1,0.00 - 0.09
3101,832,141,59.55,auto,none,Old,39.25,60.0,1,0.00 - 0.09
3123,832,142,59.76,auto,none,Old,39.25,60.0,1,0.00 - 0.09
3145,832,143,58.25,auto,none,Old,39.25,60.0,1,0.00 - 0.09
...,...,...,...,...,...,...,...,...,...,...
777450,2513,3022,57.97,transit,drive,Old,39.25,60.0,1,0.00 - 0.09
777496,2513,3027,57.69,transit,drive,New,55.00,60.0,1,0.00 - 0.04
777507,2513,3027,57.69,transit,drive,Old,39.25,60.0,1,0.00 - 0.09
777553,2513,3030,56.98,transit,drive,New,55.00,60.0,1,0.00 - 0.04


In [7]:
# Load the TAZ GeoJSON file
taz_geo = gpd.read_file("taz/WFv910_TAZ.geojson")
taz_df = pd.read_csv("taz/WFv910_TAZ.csv")
taz_geo_merged = pd.merge(taz_geo, taz_df, on="TAZID")
taz_geo_merged = taz_geo_merged[(taz_geo_merged['REMM']==1) & (taz_geo_merged['DEVPBLEPCT']>0)]
taz_geo = taz_geo_merged
#display(taz_geo_merged)

# Merge TAZ data with the extracted dataset using 'TAZID'
merged_geo = taz_geo.merge(merged_df, left_on="TAZID", right_on="TAZ_A")

# Set up Chrome WebDriver for capturing PNGs
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1200x800")
chrome_options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Custom RGB color palette for MapClass (converted to HEX)
mapclass_colors = {
    11: "#ACFA70",  # (172,250,112)
    10: "#70EA80",  # (112,234,128)
    9:  "#23D890",  # (35,216,144)
    8:  "#00C49E",  # (0,196,158)
    7:  "#00ADA4",  # (0,173,164)
    6:  "#0097A3",  # (0,151,163)
    5:  "#00829D",  # (0,130,157)
    4:  "#006D95",  # (0,109,149)
    3:  "#005886",  # (0,88,134)
    2:  "#204370",  # (32,67,112)
    1:  "#292F56",  # (41,47,86)
}

# Function to create a Folium map with custom colors and matching polygon borders
def create_map(geo_data, title, center, zoom_level, taz_p_location=None):
    """Creates a Folium map with a custom color scheme, polygon borders, and a marker at TAZ_P."""
    map_obj = folium.Map(location=taz_p_location, zoom_start=zoom_level)

    if geo_data.empty or geo_data["geometry"].isna().all():
        # If no data, add a marker indicating "No Data Available"
        folium.Marker(
            location=center,
            popup="No Data Available",
            icon=folium.Icon(color="gray")
        ).add_to(map_obj)
    else:
        # Style function to apply custom colors and matching border color
        def style_function(feature):
            mapclass = feature["properties"].get("MapClass", None)
            fill_color = mapclass_colors.get(mapclass, "#FFFFFF")  # Default to white if not found
            return {
                "fillColor": fill_color,
                "color": fill_color,  # **MATCH BORDER TO FILL COLOR**
                "weight": 0.5,  # Slight weight to avoid slivers
                "fillOpacity": 0.85
            }

        folium.GeoJson(
            geo_data,
            style_function=style_function,
            tooltip=folium.GeoJsonTooltip(fields=["MapClass"])
        ).add_to(map_obj)

    # Add a marker for the TAZ_P location if available
    if taz_p_location:
        folium.Marker(
            location=taz_p_location,
            popup=f"TAZ_P: {title}",
            icon=folium.Icon(color="red", icon="info-sign")
        ).add_to(map_obj)

    return map_obj


# Folder to store PNGs
output_folder = "output_maps"
os.makedirs(output_folder, exist_ok=True)

# Loop through each unique I value to create separate maps
for _, row_taz_rows_to_extract_df in taz_rows_to_extract_df.iterrows():

    taz_p_val = row_taz_rows_to_extract_df['taz_id']
    map_label = row_taz_rows_to_extract_df['map_label']

    for _, row in matrices_for_extraction_df.iterrows():

        auto_transit = row['AutoTransit']
        access_mode = row['AccessMode']

        subset_geo = merged_geo[
            (merged_geo["TAZ_P"] == taz_p_val) &
            (merged_geo['AutoTransit'] == auto_transit) &
            (merged_geo['AccessMode'] == access_mode)
        ]

        # Separate datasets for New and Old methods
        new_geo = subset_geo[subset_geo["Method"] == "New"]
        old_geo = subset_geo[subset_geo["Method"] == "Old"]

        # Define a consistent zoom extent based on the full dataset for `TAZ_P`
        center = [subset_geo.geometry.centroid.y.mean(), subset_geo.geometry.centroid.x.mean()]
        zoom_level = 10  # Adjust if necessary

        # Find the centroid of the TAZ polygon where TAZID == TAZ_P
        taz_p_location = None
        taz_match = taz_geo[taz_geo["TAZID"] == taz_p_val]

        if not taz_match.empty:
            taz_p_location = [
                taz_match.geometry.centroid.y.values[0],
                taz_match.geometry.centroid.x.values[0]
            ]

        # Ensure an empty GeoDataFrame is properly structured
        empty_geojson = gpd.GeoDataFrame({'MapClass': [], 'geometry': []}, geometry='geometry', crs="EPSG:4326")

        # Create maps with the correct TAZ marker
        if not new_geo.empty:
            new_map = create_map(new_geo, f"{map_label}_{auto_transit}_{access_mode}_New_Method", center, zoom_level, taz_p_location)
        else:
            print(f"No data for New Method: {map_label}, {auto_transit}, {access_mode}")
            new_map = create_map(empty_geojson, f"{map_label}_{auto_transit}_{access_mode}_New_Method (No Data)", center, zoom_level, taz_p_location)

        if not old_geo.empty:
            old_map = create_map(old_geo, f"{map_label}_{auto_transit}_{access_mode}_Old_Method", center, zoom_level, taz_p_location)
        else:
            print(f"No data for Old Method: {map_label}, {auto_transit}, {access_mode}")
            old_map = create_map(empty_geojson, f"{map_label}_{auto_transit}_{access_mode}_Old_Method (No Data)", center, zoom_level, taz_p_location)

        # Save maps as temporary HTML files
        new_map_html = f"output_html/map_temp_{map_label}_{auto_transit}_{access_mode}_new.html"
        old_map_html = f"output_html/map_temp_{map_label}_{auto_transit}_{access_mode}_old.html"
        
        new_map.save(new_map_html)
        old_map.save(old_map_html)

        # Convert HTML to PNG
        def save_as_png(html_file, output_file):
            """Renders the HTML map and saves it as a PNG image."""
            file_url = f"file://{os.path.abspath(html_file)}"
            driver.get(file_url)
            driver.save_screenshot(output_file)
            print(f"Saved: {output_file}")

        # File names with TAZ_P at the beginning
        if access_mode == 'none':
            new_png = os.path.join(output_folder, f"{map_label}_{auto_transit}_New_Method.png")
            old_png = os.path.join(output_folder, f"{map_label}_{auto_transit}_Old_Method.png")
        else:
            new_png = os.path.join(output_folder, f"{map_label}_{auto_transit}_{access_mode}_New_Method.png")
            old_png = os.path.join(output_folder, f"{map_label}_{auto_transit}_{access_mode}_Old_Method.png")

        save_as_png(new_map_html, new_png)
        save_as_png(old_map_html, old_png)

        # Clean up temporary HTML files
        os.remove(new_map_html)
        os.remove(old_map_html)

# Close the WebDriver
driver.quit()

print(f"All PNG maps saved in '{output_folder}' folder.")


C:\Users\bhereth\AppData\Local\Temp\ipykernel_21048\1245522130.py:102: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [subset_geo.geometry.centroid.y.mean(), subset_geo.geometry.centroid.x.mean()]
C:\Users\bhereth\AppData\Local\Temp\ipykernel_21048\1245522130.py:111: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.y.values[0],
C:\Users\bhereth\AppData\Local\Temp\ipykernel_21048\1245522130.py:112: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.x.values[0]


Saved: output_maps\Bountiful_auto_New_Method.png
Saved: output_maps\Bountiful_auto_Old_Method.png


C:\Users\bhereth\AppData\Local\Temp\ipykernel_21048\1245522130.py:102: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [subset_geo.geometry.centroid.y.mean(), subset_geo.geometry.centroid.x.mean()]
C:\Users\bhereth\AppData\Local\Temp\ipykernel_21048\1245522130.py:111: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.y.values[0],
C:\Users\bhereth\AppData\Local\Temp\ipykernel_21048\1245522130.py:112: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.x.values[0]


Saved: output_maps\Bountiful_transit_walk_New_Method.png
Saved: output_maps\Bountiful_transit_walk_Old_Method.png


C:\Users\bhereth\AppData\Local\Temp\ipykernel_21048\1245522130.py:102: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [subset_geo.geometry.centroid.y.mean(), subset_geo.geometry.centroid.x.mean()]
C:\Users\bhereth\AppData\Local\Temp\ipykernel_21048\1245522130.py:111: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.y.values[0],
C:\Users\bhereth\AppData\Local\Temp\ipykernel_21048\1245522130.py:112: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.x.values[0]


...

Saved: output_maps\Saratoga_auto_New_Method.png
Saved: output_maps\Saratoga_auto_Old_Method.png


C:\Users\bhereth\AppData\Local\Temp\ipykernel_21048\1245522130.py:102: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [subset_geo.geometry.centroid.y.mean(), subset_geo.geometry.centroid.x.mean()]
C:\Users\bhereth\AppData\Local\Temp\ipykernel_21048\1245522130.py:111: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.y.values[0],
C:\Users\bhereth\AppData\Local\Temp\ipykernel_21048\1245522130.py:112: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.x.values[0]


Saved: output_maps\Saratoga_transit_walk_New_Method.png
Saved: output_maps\Saratoga_transit_walk_Old_Method.png


C:\Users\bhereth\AppData\Local\Temp\ipykernel_21048\1245522130.py:102: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [subset_geo.geometry.centroid.y.mean(), subset_geo.geometry.centroid.x.mean()]
C:\Users\bhereth\AppData\Local\Temp\ipykernel_21048\1245522130.py:111: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.y.values[0],
C:\Users\bhereth\AppData\Local\Temp\ipykernel_21048\1245522130.py:112: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz_match.geometry.centroid.x.values[0]


Saved: output_maps\Saratoga_transit_drive_New_Method.png
Saved: output_maps\Saratoga_transit_drive_Old_Method.png
All PNG maps saved in 'output_maps' folder.
